In [ ]:
# Importando bibliotecas necessárias
from prophet import Prophet

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Carregando dados do CSV
df = pd.read_csv("teste.csv", parse_dates=['timestamp'])

In [ ]:
# Renomear colunas para corresponder aos requisitos do Prophet
df = df.rename(columns={"value": "y", "timestamp": "ds"})

In [ ]:
# Configurando parametros
resolution = 5 #minutos
no_of_days  = 3 # dias para análise
no_of_samples_per_day = int(24*60/resolution) # número de amostras por dia
no_of_samples = no_of_samples_per_day * no_of_days # número total de amostras

# Amplitute parameters
throughput_per_replica = 10 # capacidade por réplica

In [ ]:
# Escalar o valor para obter réplicas
df["y"] = (df["y"] / throughput_per_replica).apply(np.round) 

df.head(no_of_samples_per_day*3).plot(kind='line', x='ds', y='y')

In [ ]:
# Dividir dados em dados de treino e teste
df_train = df.head(no_of_samples_per_day*6) # dados de treino (6 dias)
df_test = df.tail(no_of_samples_per_day*4) # dados de teste (4 dias)

In [ ]:
# Criar e ajustar um modelo Prophet
model = Prophet()
model.fit(df_train)

In [ ]:
# Prever valores futuros para 10 dias
future = model.make_future_dataframe(periods=no_of_samples_per_day*10, freq='{}min'.format(resolution)) # criar dataframe futuro

forecast = model.predict(future) # prever valores futuros
forecast['forecasted_y'] = forecast['yhat'].apply(np.ceil) # arredondar para cima os valores previstos

# Plotar os componentes da previsão
model.plot_components(forecast) # plotar os componentes da previsão

In [ ]:
# Plotar os dados reais e previstos
ax = forecast.tail(no_of_samples_per_day*10).head(no_of_samples_per_day*4).plot(kind='line', x='ds', y='forecasted_y') # plotar os valores previstos
df_train.tail(no_of_samples_per_day*2).plot(x='ds', y='y', ax=ax) # plotar os dados reais

In [ ]:
# Calcular o erro quadrático médio da raiz
from sklearn.metrics import mean_squared_error

# Obter os valores reais do dataframe original
actual = df_test['y']

# Obter os valores previstos do dataframe previsto
#predicted = forecast['forecasted_y'].tail(len(df_test))
predicted = forecast['forecasted_y'].tail(no_of_samples_per_day*4)

# Calcular o rmse
rmse = np.sqrt(mean_squared_error(actual, predicted))

# Imprimir o rmse
print(f"RMSE: {rmse:.2f}; {rmse * 100 / df_test['y'].max():.2f}%")

In [ ]:
# Save the prediction in a csv file
forecast = forecast.rename(columns={"forecasted_y": "value", "ds": "timestamp"})
forecast.tail(no_of_samples_per_day*6).to_csv('prediction.csv', columns=['timestamp','value'], index=False)